# John Myles White's Brownian Motion Example

##### [OpenMendel short course](http://www.genetics.ucla.edu/courses/statgene/Mendel/)
##### Dr. Hua Zhou, [huazhou@ucla.edu](mailto: huazhou@ucla.edu)
##### Department of Biostatistics, UCLA
##### Aug 25, 2016

This example is taken from John Myles White's talk at [Multithreaded Data event](http://multithreaded.stitchfix.com/blog/2015/03/05/john-myles-white-on-julia/). This is a naive implementation of Brownian motion in R:

In [9]:
using RCall
R"""
loop <- function (n) {
    x <- 0.0
    for (i in 1:n) {
       x <- x + rnorm(1)
    }
    return(x)
}
"""

RCall.RObject{RCall.ClosSxp}


Computing a single sample from this process for n = 10,000,000 takes around 35 seconds in `R`.

In [10]:
@elapsed R"""
loop(10000000)
"""

32.843564774

Here’s a translation of that R code into Julia:

In [11]:
function loop(n)
    x = 0.0
    for i in 1:n
        x = x + randn()
    end
    return x
end

loop (generic function with 1 method)

This implementation takes only 0.07 seconds to run - a whopping 500x faster than `R`.

In [12]:
@elapsed loop(10000000)

0.070060761

Some of the main reasons why the R interpreter executes so much more slowly than the Julia compiler include:

* Numerical values in R are subject to indirection because R almost never assumes that the type of a variable is constant throughout the body of a function.

* Scalar values in R don’t exist, which imposes additional indirection when you only want to work with a single scalar value.

* Because scalar values don’t exist in R, every single addition step in the main loop of this function has to allocate a new chunk of memory in which to store a vector.

* Because R allows function calls within a function to change the semantics of almost any construct in the language, every operation must check whether its semantics are unaltered at every iteration in the loop.

So what makes Julia so efficient?

* Julia infers the types of all variables inside the body of the loop function, conditional on knowing the types of the input arguments `n`.

* With the results of type inference in hand, Julia is able to ask LLVM to generate machine code at run-time. This code corresponds to what a simple translation of the type-annotated Julia code into C would compile into.

* Julia then executes the function body using the run-time compiled code rather than interpreting the raw source code.

In [15]:
@code_lowered loop(Int)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:n], Any[Any[Any[:n,:Any,0],Any[:x,:Any,2],Any[symbol("#s52"),:Any,2],Any[:i,:Any,18]],Any[],2,Any[]], :(begin  # In[11], line 2:
        x = 0.0 # In[11], line 3:
        GenSym(0) = (Main.colon)(1,n)
        #s52 = (top(start))(GenSym(0))
        unless (top(!))((top(done))(GenSym(0),#s52)) goto 1
        2: 
        GenSym(1) = (top(next))(GenSym(0),#s52)
        i = (top(getfield))(GenSym(1),1)
        #s52 = (top(getfield))(GenSym(1),2) # In[11], line 4:
        x = x + (Main.randn)()
        3: 
        unless (top(!))((top(!))((top(done))(GenSym(0),#s52))) goto 2
        1: 
        0:  # In[11], line 6:
        return x
    end))))

In [16]:
@code_warntype loop(10)

Variables:
  n::Int64
  x::Float64
  #s52::Int64
  i::Int64
  ####r#7687#7713::UInt64
  ####rabs#7688#7714::Int64
  ####idx#7689#7715::Int64
  ####x#7690#7716::Float64
  ########_var0#7679#7685#7691#7717::UNION{BOOL,INT64}
  ############_var0#7100#7678#7680#7686#7692#7718::Float64
  ##_var0#7719::Float64

Body:
  begin  # In[11], line 2:
      x = 0.0 # In[11], line 3:
      GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64)))
      #s52 = (top(getfield))(GenSym(0),:start)::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s52::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Bool)) goto 1
      2: 
      GenSym(4) = #s52::Int64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s52::Int64,1))
      i = GenSym(4)
      #s52 = GenSym(5) # In[11], line 4:
      $(Expr(:boundscheck, false))
      unless 

In [17]:
@code_llvm loop(10)


define double @julia_loop_22420(i64) {
top:
  %1 = alloca [4 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [4 x %jl_value_t*]* %1, i64 0, i64 0
  %2 = getelementptr [4 x %jl_value_t*]* %1, i64 0, i64 2
  store %jl_value_t* inttoptr (i64 4 to %jl_value_t*), %jl_value_t** %.sub, align 8
  %3 = load %jl_value_t*** @jl_pgcstack, align 8
  %4 = getelementptr [4 x %jl_value_t*]* %1, i64 0, i64 1
  %.c = bitcast %jl_value_t** %3 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %4, align 8
  store %jl_value_t** %.sub, %jl_value_t*** @jl_pgcstack, align 8
  store %jl_value_t* null, %jl_value_t** %2, align 8
  %5 = getelementptr [4 x %jl_value_t*]* %1, i64 0, i64 3
  store %jl_value_t* null, %jl_value_t** %5, align 8
  %6 = icmp sgt i64 %0, 0
  %7 = select i1 %6, i64 %0, i64 0
  %8 = icmp eq i64 %7, 0
  br i1 %8, label %L15, label %L

L:                                                ; preds = %L12, %top
  %"#s52.0" = phi i64 [ %56, %L12 ], [ 1, %top ]
  %x.0 = phi double [ %

In [18]:
@code_native loop(10)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[11]
Source line: 2
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$40, %rsp
	movq	$4, -72(%rbp)
	movabsq	$jl_pgcstack, %rcx
	movq	(%rcx), %rax
	movq	%rax, -64(%rbp)
	leaq	-72(%rbp), %rax
	movq	%rax, (%rcx)
	xorps	%xmm0, %xmm0
	movups	%xmm0, -56(%rbp)
	xorl	%ebx, %ebx
	testq	%rdi, %rdi
	movl	$0, %eax
Source line: 3
	cmovnsq	%rdi, %rax
	xorps	%xmm0, %xmm0
	testq	%rax, %rax
	je	L335
	testq	%rdi, %rdi
	cmovnsq	%rdi, %rbx
	xorps	%xmm0, %xmm0
	movabsq	$4541175352, %r15       ## imm = 0x10EACD638
	movabsq	$4541179528, %r13       ## imm = 0x10EACE688
Source line: 4
	movabsq	$13357183728, %r14      ## imm = 0x31C2672F0
	movabsq	$jl_false, %r12
L132:	movsd	%xmm0, -80(%rbp)
	movq	(%r15), %rdi
	cmpq	$382, 16(%rdi)          ## imm = 0x17E
	jne	L186
	movabsq	$13357177264, %rax      ## imm = 0x31C2659B0
	callq	*%rax
	movq	%rax, %rdi
	movabsq	$jl_box_int64, %rax
	callq	*%rax
	j